# Inception - Digging into Spark Apps Using Spark

Apache Spark is a complex big data engine. At scale, failure become more of a norm than an exception. When your application fails, knowing what logs exist and how to take a look will help you debug and arrive at the root cause.

When writing Spark applications, there are three kinds of logs to pay attention to:

  1. Event Logs
  2. Driver Logs
  3. Executor Logs

In this notebook, we will cover digging into Spark logs.

## Analyzing Event Logs

The first half of notebook assists you in analyzing Event Log of Spark application. 

* Event Log json file is read to a dataframe.
* Useful queries are provided to obtain event related insight. 

In [3]:
// Spin up a spark session.
spark

res2: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6150d996

### Set the path of Event Log file and read the file.
A sample Event Log file is used to create this notebook. You have to set the path to your event log file accordingly. 

In [4]:
// Set the path of Event Log file.
val eventLogPath = "<your-event-log-path>"
    
// Read the Event Log file using spark json reader.
val eventLogDf = spark.read.json(eventLogPath)

eventLogPath: String = <your-event-log-path>
eventLogDf: org.apache.spark.sql.DataFrame = [App Attempt ID: string, App ID: string ... 38 more fields]

In [5]:
// Uncomment the following line if you want to print the highly nested schema of eventLogDf. 
// eventLogDf.printSchema

In [6]:
// Check the number of different events.
eventLogDf.select("Event").groupBy("Event").count.show(false)

+-----------------------------------------------------------------+-----+
|Event                                                            |count|
+-----------------------------------------------------------------+-----+
|org.apache.spark.sql.execution.ui.SparkListenerSQLExecutionEnd   |2    |
|SparkListenerTaskStart                                           |44   |
|SparkListenerBlockManagerAdded                                   |3    |
|SparkListenerJobStart                                            |3    |
|SparkListenerStageCompleted                                      |5    |
|SparkListenerJobEnd                                              |3    |
|SparkListenerApplicationEnd                                      |1    |
|com.microsoft.peregrine.spark.listeners.PlanLogEvent             |2    |
|org.apache.spark.sql.execution.ui.SparkListenerDriverAccumUpdates|2    |
|SparkListenerLogStart                                            |1    |
|SparkListenerExecutorAdded           

In [7]:
// Check User, App ID and App Name.
eventLogDf.select("User").filter(eventLogDf.col("User").isNotNull).show(false)
eventLogDf.select("App ID").filter(eventLogDf.col("App ID").isNotNull).show(false)
eventLogDf.select("App Name").filter(eventLogDf.col("App Name").isNotNull).show(false)

+--------------------+
|User                |
+--------------------+
|trusted-service-user|
+--------------------+

+------------------------------+
|App ID                        |
+------------------------------+
|application_1587879306397_0001|
+------------------------------+

+------------------------------------------+
|App Name                                  |
+------------------------------------------+
|AppName_1587879236530|
+------------------------------------------+

In [8]:
// Find Driver Logs, including url for stderr and stdout.
eventLogDf.select("Driver Logs").printSchema
eventLogDf.select("Driver Logs").filter(eventLogDf.col("Driver Logs").isNotNull).show(false)

root
 |-- Driver Logs: struct (nullable = true)
 |    |-- stderr: string (nullable = true)
 |    |-- stdout: string (nullable = true)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Driver Logs                                                                                                                                                                                                                                                                                                           |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
// Find the number of Executors and Executor Info, which includes Host ID, Log Urls and Total Cores.
eventLogDf.select("Executor ID").filter(eventLogDf.col("Executor ID").isNotNull).show()
eventLogDf.select("Executor Info").printSchema
eventLogDf.select("Executor Info").filter(eventLogDf("Executor Info").isNotNull).show(false)

+-----------+
|Executor ID|
+-----------+
|          1|
|          2|
+-----------+

root
 |-- Executor Info: struct (nullable = true)
 |    |-- Host: string (nullable = true)
 |    |-- Log Urls: struct (nullable = true)
 |    |    |-- stderr: string (nullable = true)
 |    |    |-- stdout: string (nullable = true)
 |    |-- Total Cores: long (nullable = true)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Executor Info                                                                                                                                                                                                                                                                   

In [10]:
// Show memory configuration, including Maximum Memory, Maximum Onheap Memory and Maximum Offheap Memory.
eventLogDf.select("Maximum Memory").filter(eventLogDf("Maximum Memory").isNotNull).distinct().show(false)
eventLogDf.select("Maximum Onheap Memory").filter(eventLogDf("Maximum Onheap Memory").isNotNull).distinct().show(false)
eventLogDf.select("Maximum Offheap Memory").filter(eventLogDf("Maximum Offheap Memory").isNotNull).distinct().show(false)

+--------------+
|Maximum Memory|
+--------------+
|15845975654   |
+--------------+

+---------------------+
|Maximum Onheap Memory|
+---------------------+
|15845975654          |
+---------------------+

+----------------------+
|Maximum Offheap Memory|
+----------------------+
|0                     |
+----------------------+

## Analyzing Spark App Driver and Executor Logs

The second half of notebook notebook assists you in analyzing Spark application driver or executor log.
* Driver or executor log text file is read as a dataframe.
* The dataframe is transformed that it has two columns "Timestamp" and "Message".
* A simple query is provided to filter log containing possible error message. 

### Set the path of driver or executor log file and read the file.
A sample driver stderr log file is used when creating this notebook. You have to change the path to your driver or executor log file accordingly.

In [11]:
// Set the path of log file.
val driverLogPath = "<your-driver-or-executor-log-path>"
// Read the Log file using spark text reader.
val driverLogDf = spark.read.text(driverLogPath)

driverLogPath: String = <your-driver-or-executor-log-path>
driverLogDf: org.apache.spark.sql.DataFrame = [value: string]

In [12]:
// Define a function that split the log line into timestamp and message.
def splitingLine: (String => (String, String)) = {
    // timestamp eg: "20/04/26 05:35:39"
    val timestampPattern = raw"(\d{2})/(\d{2})/(\d{2})(\s)(\d{2}):(\d{2}):(\d{2})".r
    val timestampLength = 17
    line => {
        val filler = " " * timestampLength
        if(line.length < timestampLength){
            (filler, line)            
        } else {
            val prefix = line.substring(0, timestampLength)
            prefix match {
                case timestampPattern(_*) => (prefix, line.substring(timestampLength))
                case _ => (filler, line)
            }
        }
    }
}

// Create an UDF based on the splitingLine function.
val splitingUDF = udf(splitingLine)

splitingLine: String => (String, String)
splitingUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,StructType(StructField(_1,StringType,true), StructField(_2,StringType,true)),Some(List(StringType)))

In [13]:
// Generate a driver log based dataframe with only two columns: Timestamp and Message.
val transformedDf = driverLogDf.withColumn("newCol", splitingUDF(driverLogDf("value")))
val finalDriverLogDf = transformedDf.select(transformedDf("newCol._1").as("Timestamp"), transformedDf("newCol._2").as("Message"))

finalDriverLogDf.show()

transformedDf: org.apache.spark.sql.DataFrame = [value: string, newCol: struct<_1: string, _2: string>]
finalDriverLogDf: org.apache.spark.sql.DataFrame = [Timestamp: string, Message: string]
+-----------------+--------------------+
|        Timestamp|             Message|
+-----------------+--------------------+
|                 |Container: contai...|
|                 |LogAggregationTyp...|
|                 |=================...|
|                 |      LogType:stderr|
|                 |LogLastModifiedTi...|
|                 |     LogLength:99632|
|                 |        LogContents:|
|                 |SLF4J: Class path...|
|                 |SLF4J: Found bind...|
|                 |SLF4J: Found bind...|
|                 |SLF4J: See http:/...|
|                 |SLF4J: Actual bin...|
|20/04/26 05:35:39| INFO SignalUtils...|
|20/04/26 05:35:39| INFO SignalUtils...|
|20/04/26 05:35:39| INFO SignalUtils...|
|20/04/26 05:35:40| INFO SecurityMan...|
|20/04/26 05:35:40| INFO Secu

In [14]:
// Find all the messages containning "Error" or "error". 
finalDriverLogDf.filter(finalDriverLogDf("Message").contains("error") || finalDriverLogDf("Message").contains("Error")).show(false)

+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Timestamp        |Message                                                                                                                                                                              |
+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                 |      -XX:OnOutOfMemoryError='kill %p' \                                                                                                                                             |
|20/04/26 06:26:19| INFO YarnAllocator: Executor for container container_1587879306397_0001_01_000003 exited because of a YARN event (e.g., pre-emption) and not because of an error in the runn